# Day 02.12.2021 Simulation

In [1]:
import pandas as pd
import numpy as np
from data_tools.simulation_data import SimulationDataCreator
from data_tools.simulation_run import RunSimulation
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.precision", 5)

In [2]:
# upload, clean and prepare the data
simulation_data_creator = SimulationDataCreator("/Users/tom/dev/bbp-research/configuration_files/simulation_data_creator.json")
# upload the data
data_dict = simulation_data_creator.upload_data(["market_data", "market_data_1", "view_market_data"])
# convert invoker column
data_dict = simulation_data_creator.convert_invokers(data_dict, ["market_data", "market_data_1", "view_market_data"])
# merge the data
merged_data = simulation_data_creator.merge_markets(data_dict["market_data"]["data"], data_dict["market_data_1"]["data"], "timestamp", True)
merged_data = simulation_data_creator.merge_markets(merged_data, data_dict["view_market_data"]["data"], "timestamp", True)
# # add time
merged_data = simulation_data_creator.add_time(merged_data, "timestamp")
# data_dict = simulation_data_creator.add_times(data_dict, ["market_data", "market_data_1", "view_market_data"])
# cut hours in between 10:00 to 19:00
merged_data = simulation_data_creator.keep_data_between_hours(merged_data, 9, 19, "hour")

the value counts of column invoker is:

FCI_offer_0    720757
FCI_bid_0      683806
IBI_bid_0      294108
IBI_offer_0    294108
TA1_offer_0    183453
TA1_bid_0      183452
EBS_offer_0    170324
EBS_bid_0      170324
IDB_offer_0    154616
IDB_bid_0      154616
dtype: int64
1705224, 1705224
the value counts of column invoker is:

CITIQ_BARAK_offer_0       44581
CITIQ_BARAK_bid_0         44534
CREDITQ_BARAK_offer_0     34838
CREDITQ_BARAK_bid_0       34831
JPQ_BARAK_bid_0           32918
JPQ_BARAK_offer_0         30401
UBSQ_BARAK_offer_0        23504
UBSQ_BARAK_bid_0          23481
GOLDMANQ_BARAK_bid_0      21705
GOLDMANQ_BARAK_offer_0    21661
dtype: int64
285184, 285184
the value counts of column invoker is:

TRADAIR_LP12_offer_0    218767
TRADAIR_LP12_bid_0      218767
TRADAIR_LP1_offer_0     154098
TRADAIR_LP1_bid_0       154098
TRADAIR_LP15_bid_0      133454
TRADAIR_LP15_offer_0    133454
TRADAIR_LP3_bid_0       118637
TRADAIR_LP3_offer_0     118637
TRADAIR_LP6_bid_0       116233
TRA

## arena 1 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB']
- view_brokers = none
- positon_fee = 0.00001
- max time in position = false

In [3]:
arena_1_sim_1 = RunSimulation(simulation_name = "arena 1 simulation 1", 
                              hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB'],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_1_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_1_sim_1_df = merged_data.copy()
sim_df_col_list, sim_df_np, sim_number_of_rows = arena_1_sim_1.prepaer_data_for_simulation(arena_1_sim_1_df)

In [4]:
for i in range(sim_number_of_rows):
    # current market row
    current_market_data = sim_df_np[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_1_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.00005
            lower_limit_value = best_bid_value - 0.00005
            while temp_index < sim_number_of_rows and (sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] >= upper_limit_value or sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < lower_limit_value):
                temp_index += 1
            if temp_index >= sim_number_of_rows:
                break
            else:
                arena_1_sim_1.documnet_position(current_market_data, sim_df_np[temp_index], spread_tag_inc)

In [6]:
df_to_present = arena_1_sim_1.create_simulation_summary()
df_to_present

,signal_value,timestamp_at_entering,hour_at_entering,best_offer_at_entering,best_offer_broker_at_entering,best_bid_at_entering,best_bid_broker_at_entering,best_offer_at_exiting,best_offer_broker_at_exiting,best_bid_at_exiting,best_bid_broker_at_exiting,timestamp_at_exiting,position_outcome,position_profit
0,-0.00001,1638428446583,9,1.13214,TA1_offer_0_rate,1.13215,IBI_bid_0_rate,1.13214,TA1_offer_0_rate,1.13215,IBI_bid_0_rate,1638428446585,0.00001,0.00000
1,-0.00002,1638428446636,9,1.13214,TA1_offer_0_rate,1.13216,IBI_bid_0_rate,1.13214,TA1_offer_0_rate,1.13216,FCI_bid_0_rate,1638428446637,0.00002,0.00001
2,-0.00002,1638428446685,9,1.13214,TA1_offer_0_rate,1.13218,FCI_bid_0_rate,1.13214,TA1_offer_0_rate,1.13216,IBI_bid_0_rate,1638428446686,0.00002,0.00001
3,-0.00002,1638428472661,9,1.13211,TA1_offer_0_rate,1.13213,IBI_bid_0_rate,1.13211,TA1_offer_0_rate,1.13213,FCI_bid_0_rate,1638428472663,0.00002,0.00001
4,-0.00002,1638428472699,9,1.13211,TA1_offer_0_rate,1.13213,FCI_bid_0_rate,1.13215,IBI_offer_0_rate,1.13213,FCI_bid_0_rate,1638428472748,0.00002,0.00001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.00008,1638432378194,10,1.13161,TA1_offer_0_rate,1.13169,IBI_bid_0_rate,1.13167,IBI_offer_0_rate,1.13165,FCI_bid_0_rate,1638432591282,0.00004,0.00003
96,-0.00007,1638432378429,10,1.13161,TA1_offer_0_rate,1.13168,IBI_bid_0_rate,1.13167,IBI_offer_0_rate,1.13165,FCI_bid_0_rate,1638432591282,0.00004,0.00003
97,-0.00009,1638432378770,10,1.13161,TA1_offer_0_rate,1.13170,IBI_bid_0_rate,1.13167,IBI_offer_0_rate,1.13165,FCI_bid_0_rate,1638432591282,0.00004,0.00003
98,-0.00009,1638432379020,10,1.13161,TA1_offer_0_rate,1.13170,IBI_bid_0_rate,1.13167,IBI_offer_0_rate,1.13165,FCI_bid_0_rate,1638432591282,0.00004,0.00003


In [7]:
df_to_present["position_profit"].sum()

0.0016800000000000005

## arena 2 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = [TA1, IBI]
- positon_fee = 0.00001
- max time in position = false

In [35]:
arena_2_sim_1 = RunSimulation(simulation_name = "arena 2 simulation 1", 
                              hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_2_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_2_sim_1_df = merged_data.copy()
sim_df_col_list, sim_df_np, sim_number_of_rows = arena_2_sim_1.prepaer_data_for_simulation(arena_2_sim_1_df)

In [38]:
for i in range(sim_number_of_rows):
    # current market row
    current_market_data = sim_df_np[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_2_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.00005
            lower_limit_value = best_bid_value - 0.00005
            while temp_index < sim_number_of_rows and (sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] >= upper_limit_value or sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < lower_limit_value):
                temp_index += 1
            if temp_index >= sim_number_of_rows:
                break
            else:
                arena_2_sim_1.documnet_position(current_market_data, sim_df_np[temp_index], spread_tag_inc)

tom


In [40]:
df_to_present = arena_2_sim_1.create_simulation_summary()
df_to_present

,signal_value,timestamp_at_entering,hour_at_entering,best_offer_at_entering,best_offer_broker_at_entering,best_bid_at_entering,best_bid_broker_at_entering,best_offer_at_exiting,best_offer_broker_at_exiting,best_bid_at_exiting,best_bid_broker_at_exiting,timestamp_at_exiting,position_outcome,position_profit
0,-0.00001,1638428446583,9,1.13210,UBSQ_BARAK_offer_0_rate,1.13212,JPQ_BARAK_bid_0_rate,1.13210,UBSQ_BARAK_offer_0_rate,1.13212,JPQ_BARAK_bid_0_rate,1638428446585,0.00002,0.00001
1,-0.00002,1638428446636,9,1.13210,UBSQ_BARAK_offer_0_rate,1.13212,JPQ_BARAK_bid_0_rate,1.13210,UBSQ_BARAK_offer_0_rate,1.13212,JPQ_BARAK_bid_0_rate,1638428446637,0.00002,0.00001
2,-0.00002,1638428446685,9,1.13210,UBSQ_BARAK_offer_0_rate,1.13212,JPQ_BARAK_bid_0_rate,1.13210,UBSQ_BARAK_offer_0_rate,1.13212,JPQ_BARAK_bid_0_rate,1638428446686,0.00002,0.00001
3,-0.00002,1638428472661,9,1.13208,UBSQ_BARAK_offer_0_rate,1.13207,JPQ_BARAK_bid_0_rate,1.13208,UBSQ_BARAK_offer_0_rate,1.13207,JPQ_BARAK_bid_0_rate,1638428472663,-0.00001,-0.00002
4,-0.00002,1638428472699,9,1.13208,UBSQ_BARAK_offer_0_rate,1.13207,JPQ_BARAK_bid_0_rate,1.13208,UBSQ_BARAK_offer_0_rate,1.13207,JPQ_BARAK_bid_0_rate,1638428472748,-0.00001,-0.00002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,-0.00006,1638431850299,9,1.13081,CREDITQ_BARAK_offer_0_rate,1.13081,CITIQ_BARAK_bid_0_rate,1.13081,CREDITQ_BARAK_offer_0_rate,1.13081,CITIQ_BARAK_bid_0_rate,1638431850302,0.00000,-0.00001
66,-0.00004,1638431850351,9,1.13081,CREDITQ_BARAK_offer_0_rate,1.13081,CITIQ_BARAK_bid_0_rate,1.13081,CREDITQ_BARAK_offer_0_rate,1.13081,CITIQ_BARAK_bid_0_rate,1638431850359,0.00000,-0.00001
67,-0.00003,1638431850403,9,1.13081,CREDITQ_BARAK_offer_0_rate,1.13081,CITIQ_BARAK_bid_0_rate,1.13081,CREDITQ_BARAK_offer_0_rate,1.13081,CITIQ_BARAK_bid_0_rate,1638431850405,0.00000,-0.00001
68,-0.00003,1638432000970,10,1.13101,CITIQ_BARAK_offer_0_rate,1.13099,CITIQ_BARAK_bid_0_rate,1.13101,CITIQ_BARAK_offer_0_rate,1.13099,CITIQ_BARAK_bid_0_rate,1638432000988,-0.00002,-0.00003


In [41]:
df_to_present["position_profit"].sum()

-0.0012100000000000001

## arena 3 simulation 1
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005
- hit_brokers = ["FCI", "IBI", "TA1", "IDB", "CITIQ_BARAK", "CREDITQ_BARAK","UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = None
- positon_fee = 0.00001
- max time in position = false

In [ ]:
arena_3_sim_1 = RunSimulation(simulation_name = "arena 3 simulation 1", 
                              hit_brokers = ["FCI", "IBI", "TA1", "IDB", "CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"],
                              view_brokers = [],
                              enter_conditions = "spread_tag_inc(TA1_offer, IBI_bid) < 0",
                              exit_conditions = "current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005",
                              positon_fee = 0.00001, 
                              max_time_in_position = False)
# initialize summary data frame                   
arena_3_sim_1.initialize_summary_df("increase")
# prepaer data for simulation
arena_3_sim_1_df = merged_data.copy()
sim_df_col_list, sim_df_np, sim_number_of_rows = arena_3_sim_1.prepaer_data_for_simulation(arena_3_sim_1_df)

In [ ]:
for i in range(sim_number_of_rows):
    # current market row
    current_market_data = sim_df_np[i]
    # check if entering condition holds
    if current_market_data[sim_df_col_list.index("IBI_bid_0")]:
        spread_tag_inc = current_market_data[sim_df_col_list.index("TA1_offer_0_rate")] - current_market_data[sim_df_col_list.index("IBI_bid_0_rate")]
        if spread_tag_inc < 0.0:
            best_offer_value, best_offer_broker, best_bid_value, best_bid_broker = arena_3_sim_1.find_bests_for_hit_brokers(current_market_data)
            temp_index = i + 1
            upper_limit_value = best_offer_value + 0.00005
            lower_limit_value = best_bid_value - 0.00005
            while temp_index < sim_number_of_rows and (sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] >= upper_limit_value or sim_df_np[temp_index][sim_df_col_list.index("best_bid_hit_rate")] < lower_limit_value):
                temp_index += 1
            if temp_index >= sim_number_of_rows:
                break
            else:
                arena_3_sim_1.documnet_position(current_market_data, sim_df_np[temp_index], spread_tag_inc)

In [ ]:
df_to_present = arena_3_sim_1.create_simulation_summary()
df_to_present

In [ ]:
df_to_present["position_profit"].sum()

## arena 1 simulation 2
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005 AND 3 horese in the offer direction
- hit_brokers = ['FCI', 'IBI', 'TA1', 'IDB']
- view_brokers = LP's
- positon_fee = 0.00001
- max time in position = false

## arena 2 simulation 2
- entering position condition: spread_tag_inc(TA1_offer, IBI_bid) < 0
- exiting position condition: current_best_bid >= best_offer_at_signal + 0.00005 or current_best_bid < best_bid_at_signal - 0.00005 AND 3 horese in the offer direction
- hit_brokers = ["CITIQ_BARAK", "CREDITQ_BARAK", "UBSQ_BARAK", "GOLDMANQ_BARAK", "JPQ_BARAK"]
- view_brokers = [TA1, IBI] + LP's
- positon_fee = 0.00001
- max time in position = false